Training a model on top of VGG16 for image quality 

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import cv2
import random 
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

Creating the traning data, resizing all images to 200x200 and using color(R,G,B)

In [5]:
DATADIR = "/Users/macbook/OCR/W2"
CATEGORIES = ["Good", "Bad"]
training_data = []
IMG_SIZE = 200         # This is the size we are using 

def create_training_data():
    for category in CATEGORIES:                 # loop threw each folder with in W2 folder  
        path = os.path.join(DATADIR,category)   # Path to folder 
        class_num = CATEGORIES.index(category)  # labeling the data based on folder 
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR) # converts the image to an array 
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))   # Resize to normalize data size 
                training_data.append([new_array, class_num]) # adds it to are traning data with the label 
            except Exception as e:
                pass
            
create_training_data()

Shuffling the data before feeding it to the CNN

In [6]:
random.shuffle(training_data)

Separating the features from the labels and converting them to a np array 

In [14]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

We start by normalizing the data by scaling it, min is 0 and max is 255 for pixel data 
So we will divide it by 255, Keras also has a built in function to do this

In [15]:
X = X/255 #255 pixels max for pixel data 

Instantiate a VGG16 model that is preloaded with weights, 

We tell it the image size and that the images will be in color(3)

include_top = False will not include the classification layer, we will add one ourselves

In [7]:
IMG_SIZE = 200
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
VGG16_MODEL=tf.keras.applications.VGG16(input_shape = IMG_SHAPE,
                                               include_top = False,
                                               weights = 'imagenet')

Summary of the layers VGG16 includes 

In [3]:
VGG16_MODEL.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 212, 212, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 212, 212, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 212, 212, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 106, 106, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 106, 106, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 106, 106, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 53, 53, 128)       0     

We are frezzing the VGG16 model so that way the weights in the given model will not update 
Also including 2 more layers, one being our output layer

In [19]:
VGG16_MODEL.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2,activation='softmax')

Will convert to a sequential model and combine the last two layer we made 

In [20]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])

Compile our model using an 'adam' optimizer and 'sparese categorical crossentropy' for the loss

In [21]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

Time to fit the model with 5 epochs 

In [22]:
model.fit(X, y, batch_size = 12, epochs = 5, validation_split = .1)

Train on 3361 samples, validate on 374 samples
Epoch 1/5
3361/3361 [==============================] - 1358s 404ms/sample - loss: 0.2879 - accuracy: 0.8979 - val_loss: 0.1866 - val_accuracy: 0.9439
Epoch 2/5
3361/3361 [==============================] - 1354s 403ms/sample - loss: 0.1756 - accuracy: 0.9405 - val_loss: 0.1569 - val_accuracy: 0.9465
Epoch 3/5
3361/3361 [==============================] - 1344s 400ms/sample - loss: 0.1536 - accuracy: 0.9458 - val_loss: 0.1405 - val_accuracy: 0.9465
Epoch 4/5
3361/3361 [==============================] - 1177s 350ms/sample - loss: 0.1399 - accuracy: 0.9500 - val_loss: 0.1308 - val_accuracy: 0.9465
Epoch 5/5
3361/3361 [==============================] - 1112s 331ms/sample - loss: 0.1309 - accuracy: 0.9527 - val_loss: 0.1221 - val_accuracy: 0.9519


In [23]:
model.save("VGG16_v1")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: VGG16_v1/assets


In [30]:
DATADIR2 = "/Users/macbook/OCR/testDataGood"

test_data = []
IMG_SIZE = 200         # This is the size we are using 

def create_test_data():
    path = os.path.join(DATADIR2)   # Path to folder 

    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR) # converts the image to an array 
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))   # Resize to normalize data size 
            test_data.append([new_array]) # adds it to are traning data with the label 
        except Exception as e:
            pass
            
create_test_data()

In [31]:
test_data = np.array(test_data).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
test_data = test_data/255

predictions = model.predict_classes(test_data)


In [32]:
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0]
